In [4]:
import pandas as pd
import json
import re


rep = {" ":"", "\"":"", ",":""}

rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))

str = ""
list1 = []
list2=[]

matches = ["[", "]","\n","\t"]
file = open("sampleLOB_50.txt")

count = 0
for f in file:
    count +=1
    line = f.strip()
    f_line = pattern.sub(lambda m: rep[re.escape(m.group(0))], line)
    if any(x in f_line for x in matches):
        continue
    else:
#         str+=f_line
#         print (str)
        list1.append(f_line)

list1 = list(filter(lambda a: a != {" "}, list1))
print(count)

for i in list1:
    print(i)
    
# print(str)
# list1 = str.split(" ")
# print(list1[0])
# for i in str:
#     if(i!=None and i!=" " and i!="\n" and i!="\t"):
#         list2.append(i)
# print(len(list1))
# print(list1[0])
# print(list2[0])

101
 
      t i m e   
 
     0 . 0  
 
 
          b i d   
 
 
 
 
          a s k   
 
 
 
 
 
      t i m e   
 
     0 . 0 1 6  
 
 
          b i d   
 
 
 
 
          a s k   
 
 
 
 
 
      t i m e   
 
     0 . 0 3 2  
 
 
          b i d   
 
 
 
 
          a s k   
 
 
 
 
 
      t i m e   
 
     0 . 0 4 8  
 
 
          b i d   
 
 
 
 
          a s k   
 
 
 
 
 
      t i m e   
 
 


In [3]:
with open('sampleLOB_50.txt') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        current_line = pattern.sub(lambda m: rep[re.escape(m.group(0))], lines[i].strip())
        print(current_line)

ÿþ[ 
 
      t i m e   
 
     0 . 0  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 1 6  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 3 2  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
     0 . 0 4 8  
 
     [ 
 
          b i d   
 
         [ ] 
 
     ]  
 
     [ 
 
          a s k   
 
         [ ] 
 
     ] 
 
 ] 
 
 [ 
 
      t i m e   
 
 
